In [8]:
import cv2
import pandas as pd

In [9]:
video_folder = 'videos'
csv_file = 'new_df.csv'
frames_folder = 'frames_pose'

In [10]:
data = pd.read_csv("new_df.csv")
data

,label,youtube_id,time_start,time_end,split
0,tap dancing,GcluCxjiSjI,30,40,train
1,tap dancing,5FuYP_l6J-s,4,14,train
2,tap dancing,5AAYAhVcZNM,61,71,train
3,tap dancing,fPZb1IB-U_k,19,29,train
4,tap dancing,k9edXWpg64E,38,48,train
...,...,...,...,...,...
613,tango dancing,hC7lESvyJt8,82,92,train
614,tango dancing,Pgl0MeU6Xqs,13,23,train
615,tango dancing,UyQ5vHj0fNU,107,117,train
616,tango dancing,S2sn-1HF2LY,3,13,train


In [11]:
frames_df = pd.DataFrame()

In [12]:
for index, row in data.iterrows():
    
        # Полный путь к видео
        video_path = f"{video_folder}/{row['youtube_id']}.mp4"
        
        # Загрузка видео
        video = cv2.VideoCapture(video_path)
        
        # Установка позиции в начало интересующего сегмента видео
        video.set(cv2.CAP_PROP_POS_MSEC, row['time_start'] * 1000)
        
        # Интервал сохранения кадров в миллисекундах (здесь каждую секунду)
        frame_interval = 1000
        
        # Чтение и сохранение кадров
        frame_count = 0
        current_time = row['time_start']
        while video.isOpened() and current_time <= row['time_end']:
            ret, frame = video.read()
            if not ret:
                break
            
            # Сохранение кадра в файл
            frame_path = f"{frames_folder}/{index}_{frame_count}.jpg"
            cv2.imwrite(frame_path, frame)
            frame_count += 1
            current_time += .05
            video.set(cv2.CAP_PROP_POS_FRAMES, current_time * video.get(cv2.CAP_PROP_FPS))
            new_frame = pd.DataFrame({
                'video': row['youtube_id'],
                'frame_path': [frame_path],
                'label': row['label']
                })
            frames_df = pd.concat([frames_df, new_frame], ignore_index=True)
        video.release()

In [13]:
frames_df

,video,frame_path,label
0,GcluCxjiSjI,frames_pose/0_0.jpg,tap dancing
1,GcluCxjiSjI,frames_pose/0_1.jpg,tap dancing
2,GcluCxjiSjI,frames_pose/0_2.jpg,tap dancing
3,GcluCxjiSjI,frames_pose/0_3.jpg,tap dancing
4,GcluCxjiSjI,frames_pose/0_4.jpg,tap dancing
...,...,...,...
121497,LT-e_wj6d9w,frames_pose/617_195.jpg,tango dancing
121498,LT-e_wj6d9w,frames_pose/617_196.jpg,tango dancing
121499,LT-e_wj6d9w,frames_pose/617_197.jpg,tango dancing
121500,LT-e_wj6d9w,frames_pose/617_198.jpg,tango dancing


In [14]:
frames_df.to_csv('frames_pose.csv',index=False)